In [9]:
import pandas as pd
import os
import cv2
import numpy as np  

In [2]:
dataset_root_folder = "../DubaiDataset"
for path, subdirs, files in os.walk(dataset_root_folder):
  folder = os.path.basename(path)
  #print("FOLDER:", folder)
  if folder == "masks":
    print("PATH:", path)
    #print("FILES:", files)

    for image_name in files:
        if image_name.endswith(".png"):
            #print("PNG:", image_name) 
            a = True

     

PATH: ../DubaiDataset/Tile 8/masks
PATH: ../DubaiDataset/Tile 1/masks
PATH: ../DubaiDataset/Tile 6/masks
PATH: ../DubaiDataset/Tile 7/masks
PATH: ../DubaiDataset/Tile 5/masks
PATH: ../DubaiDataset/Tile 2/masks
PATH: ../DubaiDataset/Tile 3/masks
PATH: ../DubaiDataset/Tile 4/masks


In [3]:
image_patch_size = 256

In [4]:
image = cv2.imread("../DubaiDataset/Tile 1/images/image_part_001.jpg",1)

In [5]:
image.shape # (644,797,3) is (height, width, channels)

(644, 797, 3)

### Read All The Images and Then Crop and Patchify Them

In [6]:
def extract_patches(image, patch_size, stride=None):
    if stride is None:
        stride = patch_size

    h, w = image.shape[:2]
    patches = []

    for y in range(0, h - patch_size + 1, stride):
        for x in range(0, w - patch_size + 1, stride):
            patch = image[y:y+patch_size, x:x+patch_size]
            patches.append(patch)

    return np.array(patches)

In [7]:
image_dataset = []
mask_dataset = []
        
for tile_id in range(1, 8):
    img_dir = os.path.join(dataset_root_folder, f"Tile {tile_id}", "images")
    mask_dir = os.path.join(dataset_root_folder, f"Tile {tile_id}", "masks")
    if not os.path.exists(img_dir) or not os.path.exists(mask_dir):
        continue

    for fname in sorted(os.listdir(img_dir)):
        if not fname.endswith(".jpg"):
            continue

        img_path = os.path.join(img_dir, fname)
        mask_path = os.path.join(mask_dir, fname.replace(".jpg", ".png"))

        image = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        if image is None or mask is None:
            continue

        h, w = image.shape[:2]
        size_x = (w // image_patch_size) * image_patch_size
        size_y = (h // image_patch_size) * image_patch_size

        image = image[0:size_y, 0:size_x]
        mask = mask[0:size_y, 0:size_x]

        image_patches = extract_patches(image, image_patch_size)
        mask_patches = extract_patches(mask, image_patch_size)

        for img_p, msk_p in zip(image_patches, mask_patches):
            image_dataset.append(img_p.astype(np.float32) / 255.0)
            mask_dataset.append(msk_p)


In [13]:
image_dataset

[array([[[0.7882353 , 0.7647059 , 0.74509805],
         [0.88235295, 0.85882354, 0.8392157 ],
         [0.94509804, 0.9098039 , 0.89411765],
         ...,
         [0.89411765, 0.85882354, 0.8235294 ],
         [0.85882354, 0.83137256, 0.79607844],
         [0.8784314 , 0.8509804 , 0.8156863 ]],
 
        [[0.8862745 , 0.8627451 , 0.84313726],
         [0.99607843, 0.972549  , 0.9529412 ],
         [1.        , 1.        , 0.9843137 ],
         ...,
         [0.93333334, 0.8980392 , 0.8627451 ],
         [0.92941177, 0.9019608 , 0.8666667 ],
         [0.9529412 , 0.91764706, 0.88235295]],
 
        [[0.972549  , 0.94509804, 0.93333334],
         [1.        , 0.9882353 , 0.9764706 ],
         [1.        , 1.        , 0.9843137 ],
         ...,
         [0.93333334, 0.89411765, 0.8666667 ],
         [0.92156863, 0.89411765, 0.85882354],
         [0.9019608 , 0.8666667 , 0.83137256]],
 
        ...,
 
        [[0.88235295, 0.8627451 , 0.8666667 ],
         [0.81960785, 0.8039216 , 0.8    